**Time Independent Hedging**

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



**Ordinary Least Square Method**

In [32]:
import numpy as np
import yfinance as yf
import statsmodels.api as sm
from sklearn.decomposition import PCA
import warnings

# Suppress the FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

def download_stock_data(stocks, start_date, end_date):
    """Download stock price data for multiple stocks."""
    data = yf.download(stocks, start=start_date, end=end_date)['Close']
    data = data.dropna()  # Remove rows with missing values
    return data

def calculate_log_returns(data):
    """Calculate log returns for stock prices."""
    return np.log(data / data.shift(1)).dropna()

def calculate_hedge_ratio_ols_multi(y, X):
    """Calculate hedge ratios using OLS with multiple stocks."""
    X = sm.add_constant(X)  # Add constant for intercept
    model = sm.OLS(y, X).fit()
    return model.params[1:]  # Hedge ratios (exclude constant)



# Select multiple stocks for hedging and time period
stocks = ["AAPL","AMZN" , "GOOGL", "MSFT"]   # Example: tech stocks
hedged_stock = "AMZN"  # Stock to be hedged
hedging_stocks = [s for s in stocks if s != hedged_stock]
start_date = "2023-01-01"
end_date = "2023-12-01"

# Download stock price data
data = download_stock_data(stocks, start_date, end_date)

# Calculate log returns for all stocks
log_returns = calculate_log_returns(data)

# Separate the dependent (target) and independent (hedging) variables
y = log_returns[hedged_stock]
X = log_returns[hedging_stocks]
print(log_returns)
# Calculate hedge ratios
hedge_ratios_ols = calculate_hedge_ratio_ols_multi(y, X)


# Display results
print(f"Hedging Stock: {hedged_stock}")
print(f"Hedging Using: {hedging_stocks}")
print(f"Hedge Ratios using OLS: {dict(zip(hedging_stocks, hedge_ratios_ols))}")


[*********************100%***********************]  4 of 4 completed

Ticker          AAPL      AMZN     GOOGL      MSFT
Date                                              
2023-01-04  0.010262 -0.007955 -0.011738 -0.044729
2023-01-05 -0.010661 -0.024012 -0.021575 -0.030086
2023-01-06  0.036133  0.034992  0.013138  0.011717
2023-01-09  0.004080  0.014760  0.007755  0.009689
2023-01-10  0.004447  0.028327  0.004534  0.007588
...              ...       ...       ...       ...
2023-11-24 -0.007029  0.000204 -0.013083 -0.001112
2023-11-27 -0.000948  0.006724 -0.002051  0.003122
2023-11-28  0.003209 -0.004750  0.005775  0.010745
2023-11-29 -0.005424 -0.004841 -0.016239 -0.010111
2023-11-30  0.003058 -0.001573 -0.018392  0.000158

[229 rows x 4 columns]
Hedging Stock: AMZN
Hedging Using: ['AAPL', 'GOOGL', 'MSFT']
Hedge Ratios using OLS: {'AAPL': 0.030957308152621855, 'GOOGL': 0.4538702905768392, 'MSFT': 0.46402307985203434}


**Variance Minimization Method**

In [28]:
import numpy as np
import pandas as pd
import yfinance as yf

def download_data(stocks, start_date, end_date):
    """
    Download adjusted closing prices for the given stocks from Yahoo Finance.
    """
    data = yf.download(stocks, start=start_date, end=end_date)['Close']
    return data.dropna()  # Remove rows with missing values

def calculate_log_returns(prices):
    """
    Calculate daily log returns for the given price data.
    """
    return np.log(prices / prices.shift(1)).dropna()

def compute_hedging_ratios(target_returns, hedging_returns):
    """
    Compute the hedging ratios for a target stock using multiple hedging instruments.
    
    Parameters:
    - target_returns: Returns of the stock you want to hedge (r1)
    - hedging_returns: Returns of the hedging instruments (matrix of r_2, r_3, ..., r_n)

    Returns:
    - hedge_ratios: The optimal hedge ratios for the hedging instruments
    """
    # Covariance vector between target and hedging instruments
    cov_with_target = np.cov(target_returns, hedging_returns, rowvar=False)[0, 1:]
    
    # Covariance matrix of hedging instruments
    cov_hedgers = np.cov(hedging_returns, rowvar=False)
    
    # Hedge ratios: Var(r_H)^{-1} * Cov(r1, r_H)
    hedge_ratios = np.linalg.inv(cov_hedgers).dot(cov_with_target)
    
    return hedge_ratios

# Stock data: Set the tickers of the stocks to be included
stocks = ["AAPL","AMZN" , "GOOGL", "MSFT"]  # Example tickers: Apple, Microsoft, Google, Amazon
target_stock = "AMZN"  # The stock you want to hedge (target)

# Set the time period for the data
start_date = "2023-01-01"
end_date = "2023-12-01"

# Step 1: Download stock price data
prices = download_data(stocks, start_date, end_date)

# Step 2: Calculate daily log returns
log_returns = calculate_log_returns(prices)
print(log_returns)
# Step 3: Separate target (dependent) and hedging (independent) variables
target_returns = log_returns[target_stock]
hedging_returns = log_returns.drop(columns=[target_stock]).values  # Matrix of r_2, r_3, ..., r_n

# Step 4: Compute hedging ratios
hedge_ratios = compute_hedging_ratios(target_returns, hedging_returns)

# Display results
hedging_stock_names = [stock for stock in stocks if stock != target_stock]
hedge_ratios_dict = dict(zip(hedging_stock_names, hedge_ratios))

print("Hedging Target:", target_stock)
print("Hedging Using:", hedging_stock_names)
print("Hedge Ratios:")
for stock, ratio in hedge_ratios_dict.items():
    print(f"  {stock}: {ratio:.4f}")

[*********************100%***********************]  4 of 4 completed

Ticker          AAPL      AMZN     GOOGL      MSFT
Date                                              
2023-01-04  0.010262 -0.007955 -0.011738 -0.044729
2023-01-05 -0.010661 -0.024012 -0.021575 -0.030086
2023-01-06  0.036133  0.034992  0.013138  0.011717
2023-01-09  0.004080  0.014760  0.007755  0.009689
2023-01-10  0.004447  0.028327  0.004534  0.007588
...              ...       ...       ...       ...
2023-11-24 -0.007029  0.000204 -0.013083 -0.001112
2023-11-27 -0.000948  0.006724 -0.002051  0.003122
2023-11-28  0.003209 -0.004750  0.005775  0.010745
2023-11-29 -0.005424 -0.004841 -0.016239 -0.010111
2023-11-30  0.003058 -0.001573 -0.018392  0.000158

[229 rows x 4 columns]
Hedging Target: AMZN
Hedging Using: ['AAPL', 'GOOGL', 'MSFT']
Hedge Ratios:
  AAPL: 0.0310
  GOOGL: 0.4539
  MSFT: 0.4640
